Let's import and clean the third sheet which is `CustomerDemographic`.

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [8]:
df3 = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name=3, header=1)

In [10]:
df3.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


There's a column named `default` which has corrupted data points, let's drop that first. 

In [11]:
del df3['default']

In [15]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   int64         
 1   first_name                           4000 non-null   object        
 2   last_name                            3875 non-null   object        
 3   gender                               4000 non-null   object        
 4   past_3_years_bike_related_purchases  4000 non-null   int64         
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            3494 non-null   object        
 7   job_industry_category                3344 non-null   object        
 8   wealth_segment                       4000 non-null   object        
 9   deceased_indicator                   4000 non-null   object        
 10  owns_car    

In [88]:
# check if all the columns are unique
cols = df3.columns
for col in cols:
    print(f'{col}\t --->{df3[col].is_unique}')
    

customer_id	 --->True
first_name	 --->False
last_name	 --->False
gender	 --->False
past_3_years_bike_related_purchases	 --->False
DOB	 --->False
job_title	 --->False
job_industry_category	 --->False
wealth_segment	 --->False
deceased_indicator	 --->False
owns_car	 --->False
tenure	 --->False


Now let's look at the missing data rules that i mentioned in **TASK 1.1**.

1. ***Remove the rows with missing values:***
One option is to remove the rows that contain missing values. However, this should be done with caution, as removing too many rows can significantly reduce the amount of data available for analysis. If the missing values are only a small percentage of the total dataset, it may be acceptable to remove them.

2. ***Impute missing values:*** Another option is to impute the missing values with estimated or predicted values. There are several methods for imputing missing values, such as mean or median imputation, mode imputation, and regression imputation. Mean or median imputation can be used for numeric columns like DOB, while mode imputation can be used for categorical columns like job_titles.

3. ***Create a separate category for missing values:*** Depending on the type of missing data, you may also choose to create a separate category for the missing values. This is typically done for categorical variables, where the missing value is coded as a separate category, such as "unknown" or "not applicable"


In [96]:
# missing values in %
df3.isnull().mean()*100

customer_id                             0.000
first_name                              0.000
last_name                               3.125
gender                                  0.000
past_3_years_bike_related_purchases     0.000
DOB                                     0.000
job_title                              12.650
job_industry_category                  16.400
wealth_segment                          0.000
deceased_indicator                      0.000
owns_car                                0.000
tenure                                  2.175
dtype: float64

ok, **`job_title` and `job_industry_category`** has the major missing data points. Here we can drop column with the most missing values by using the **`thresh=` in `dropna`** method and by default it is set to rows. 

I'm only going to keep those columns that have minimum of **70% non-NaN** values.

In [35]:
df3 = df3.dropna(axis=1, how='all', thresh=(len(df3)*0.7)).copy()

In [66]:
# Let's start imputing missing data. for most of the cases mode() is a better option for filling missing data.
mode_DOB = df3['DOB'].mode()[0]
df3['DOB'].fillna(mode_DOB, inplace=True)

In [67]:
df3['DOB'].isnull().sum()

0

**job_title & job_industry_category** both are missing about **12.65% & 16.4%** of data respectively, using **ffill or bfill** to fill in missing values can be a useful approach in some cases, it is generally not recommended as the default method. Instead, it is best to use a method that is appropriate for the specific dataset and analysis.

here i'm going to fill in the missing values with the mode of the non-missing values in the column. This approach assumes that the mode is a reasonable estimate for the missing values and may be appropriate if the missing values are distributed similarly to the non-missing values.

In [101]:
# fillna for job_title
mode_job_title = df3['job_title'].mode()[0]
df3['job_title'].fillna(mode_job_title, inplace=True)

In [104]:
df3['job_title'].isnull().sum()

0

In [108]:
# fillna for job_industry_category
mode_job_indus = df3['job_industry_category'].mode()[0]
df3['job_industry_category'].fillna(mode_job_indus, inplace=True)

In [109]:
df3['job_industry_category'].isnull().sum()

0

In [121]:
# for tenure we're going to fill missing with a mean or median
mean_tenure = round(df3['tenure'].mean())
df3['tenure'].fillna(mean_tenure, inplace=True)

In [122]:
df3['tenure'].isnull().sum()

0

In [125]:
# Let's format the data 
df3['gender'].value_counts()

Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: gender, dtype: int64

In [135]:
name_format = {'U': 'Unidentified', 'Femal': 'Female', 'M': 'Male', 'F': 'Female'}
df3['gender'] = df3['gender'].replace(name_format)

Let's set standard **Boolean values** for columns **deceased_indicator, owns_car**

In [138]:
# deceased_indicator
df3['deceased_indicator'].replace({'N': False, 'Y': True}, inplace=True)

In [139]:
# owns_car
df3['owns_car'].replace({'Yes': True, 'No': False}, inplace=True)

In [141]:
df3

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,False,True,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,False,True,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,False,True,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,Business Systems Development Analyst,IT,Mass Customer,False,False,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Manufacturing,Affluent Customer,False,True,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,False,False,19.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,False,True,1.0
3997,3998,Sarene,Woolley,Unidentified,60,1978-01-30,Assistant Manager,IT,High Net Worth,False,False,11.0
3998,3999,Patrizius,NaN,Male,11,1973-10-24,Business Systems Development Analyst,Manufacturing,Affluent Customer,False,True,10.0


In [143]:
df3.to_csv('TASK 1-CustomerDemographic.csv', index=False)